# Data Cleaning

The data needs to be cleaned before we start preparing them for training models. The most important task here is to select a single answer for a question, because there are multiple answers available.

The answers are in the form of character ranges in the news text. On some exploration, several issues were found with the character ranges. Most notably, the range starts and/or ends in the middle of a word, which makes the answer unreadable/incorrect. There are also a few cases where the range extends one or two characters in a different paragraph.

The character ranges need to be updated to include entire words and not extend to different paragraphs.

In [1]:
import numpy as np
import pandas as pd
import random
import utils
import re

In [3]:
# Reading the data
data = pd.read_csv('data/newsqa-data-formatted.csv')
data.head()

,story_id,question,answer_char_ranges,is_answer_absent,is_question_bad,validated_answers
0,42d01e187213e86f5fe617fe32e716ff7fa3afc4,What was the amount of children murdered?,294:297|None|None,0.0,0.0,"{""none"": 1, ""294:297"": 2}"
1,c48228a52f26aca65c31fad273e66164f047f292,Where was one employee killed?,34:60|1610:1618|34:60,0.0,0.0,NaN
2,c65ed85800e4535f4bbbfa2c34d7d9630358d303,who did say South Africa did not issue a visa ...,103:127|114:127|839:853,0.0,0.0,"{""839:853"": 1, ""103:127"": 2}"
3,0cf66b646e9b32076513c050edf32a799200c3c2,How many years old was the businessman?,538:550|538:550,0.0,0.0,NaN
4,13012604e3203c18df09289dfedd14cde67cf40b,What frightened the families?,690:742|688:791|630:646,0.0,0.0,"{""688:791"": 2, ""690:742"": 1}"


In [4]:
# We don't need is_answer_absent and is_question_bad columns
data = data.drop(['is_answer_absent', 'is_question_bad'], axis = 1)

In [5]:
# Reading all stories as dictionary
STORY_PATH = 'cnn/stories/'
STORY_EXT = '.story'

story_ids = list(set(data['story_id']))
NEWS_STORIES = {}
cnt = 0
tot = len(story_ids)

for sid in story_ids:
    with open(STORY_PATH + sid + STORY_EXT, 'r') as file:
        NEWS_STORIES[sid] = file.read()
    cnt += 1
    utils.drawProgressBar(cnt, tot)

Progress: [====================] 11939/11939

In [6]:
def adjust_answer_range(story_id, answer_range):
    '''
    Checks if answer range starts or ends in the middle of a words
    and returns the correct answer range along with answer text
    
    Parameters
    ------------
    story_id: str
              The story id where the answer is present
    
    answer_range: str
                  The answer range to check
                  
    Returns
    -----------
    [start_idx, end_idx]
    '''
    # If answer is not available, denote it as -1
    if answer_range == 'None':
        return [-1, -1]
    
    story = NEWS_STORIES[story_id]
    
    # Check for errors in answer
    if len(answer_range.split(':')) == 1:
        return [-1, -1]
    
    start_idx, end_idx = answer_range.split(':')
    start_idx, end_idx = int(start_idx), int(end_idx)
    
    # Moves back start_idx to the start of a word
    while start_idx != 0 and not utils.is_whitespace(story[start_idx - 1]) and not utils.is_punct(story[start_idx - 1]):
        start_idx = start_idx - 1
    
    # Some ranges end with a punctuation or a whitespace
    if utils.is_whitespace(story[end_idx - 1]) or utils.is_punct(story[end_idx - 1]):
        end_idx = end_idx - 1
    
    # Moves end_idx to the end of a word
    while not utils.is_whitespace(story[end_idx]) and not utils.is_punct(story[end_idx + 1]):
        end_idx = end_idx + 1
        
    # There are some answers with \n at the end followed by a letter
    # The answer will not be in two different paragraphs
    answer_text = story[start_idx:end_idx]
    answer_para = re.split('\n', answer_text)
    
    if len(answer_para[-1]) > len(answer_para[0]):
        start_idx = end_idx - len(answer_para[-1])
        answer_text = answer_para[-1]
    else:
        end_idx = start_idx + len(answer_para[0])
        answer_text = answer_para[0]
    
    return [start_idx, end_idx]

In [7]:
def get_answer(qa_details):
    '''
    A function that selects an answer for a question
    
    > If validated answers are available, the one with most votes is selected
    > If there's a tie in validated answer votes or if validated answer is not
      available, the most frequent answer is selected
    > If there's a tie here too, a random answer is selected
    
    Parameters
    ------------
    qa_details: Pandas Series
                Details of a QA pair, must include story_id, question, 
                answer_char_ranges, validated_answers as index
    '''
    # If validated answers are available, select the one with most votes
    if qa_details['validated_answers'] is not np.nan:
        validated_answers = eval(qa_details['validated_answers'])
        
        # Get the answers with maximum votes
        max_vote_ans = utils.get_max_keys(validated_answers)
        
        # Check for ties
        if len(max_vote_ans) == 1:
            return adjust_answer_range(qa_details['story_id'], max_vote_ans[0])
    
    # If validated answers are not available or if there is a tie in validated answers
    # Get all available answers
    answers = re.split(',|\|', qa_details['answer_char_ranges'])
    
    # If there is just one answer
    if len(answers) == 1:
        return adjust_answer_range(qa_details['story_id'], answers[0])
    
    # Get counts of each answer
    answer_freq = utils.get_frequency(answers)
    max_vote_ans = utils.get_max_keys(answer_freq)
    
    if len(max_vote_ans) == 1:
        return adjust_answer_range(qa_details['story_id'], max_vote_ans[0])
    
    # If there is a tie for multiple answers, return a random answer
    return adjust_answer_range(qa_details['story_id'], random.choice(answers))

In [8]:
# --- Takes a few seconds to run ---
# Select one answer range among multiple answers
data[['start_idx', 'end_idx']] = data.apply(get_answer, axis = 1, result_type = 'expand')

In [9]:
data.sample(n = 5)

,story_id,question,answer_char_ranges,validated_answers,start_idx,end_idx
80736,cf08d87683e8c03289b7dfd87c60d98246270be2,How many were arrested in Seattle?,1980:1997|1980:1983|1980:1983,NaN,1980,1983
64072,01d5b7cf16d87db5a2bed267d83aac584f58f325,What does Michael Steele accuse Dems of?,5101:5123|5503:5534|None,"{""none"": 1, ""5503:5534"": 2}",5503,5537
48385,fb2cd7b00d3438e64b85341b757d2716348e3631,How long is the film?,None|2416:2429|2416:2429,NaN,2416,2428
3002,f6ab014104858f8bcef436218232981f2d370197,What does the video show?,"107:149,65:86,92:107|231:263|231:263",NaN,231,262
53644,3ac7f16b3dc00d676e513c9fb0b5be73371874d4,Who is the USDA spokesman?,742:754|742:754,NaN,740,753


In [10]:
# Save the cleaned data
data.to_csv('data/newsqa-dataset-cleaned.csv', index = False)
utils.save_pickle('data/news_stories.pkl', NEWS_STORIES)

Variable successfully saved in news_stories.pkl
